In [ ]:
#Part zero: import packages
import numpy
import pandas
import re
import math #was requested to use math to floor aliquot numbers
from typing import Union
import datetime
import logging
import sys
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)  # Clear existing handlers

logging.basicConfig(level=logging.WARNING, stream=sys.stdout)

def main():
    input_filepath="../data/input/"
    input_filename="22-535 sample tracking.csv"#
    import_style="DF" # Other implemented options are "SF" and "Fw_import"
    export_style="Sample_map" #Implemented options are "Fw_import" and "Sample_map"
    output_filepath="../data/exports/"
    output_filename=f"TEST_{export_style}.csv" #Can be left blank as well
    row_offset=1 #Determines which row will be the column names of the pandas dataframe
    intermediate_dataframe=read_file(input_dataframe_filepath=input_filepath, input_data_filename=input_filename, import_style=import_style, row_offset=row_offset) #The return type is called intermediate_dataframe because another dataframe is created in .write() that is actually exported 
    export_dataframe(input_dataframe=intermediate_dataframe, output_filepath=output_filepath+output_filename, output_filename=output_filename, style=export_style)

def read_file(input_dataframe_filepath: str, input_data_filename:str, import_style:str, row_offset:Union[int, None]=None) -> pandas.DataFrame: #alternate datatypes could be pandas dataframes, both for the lists and each element
    """Reads in a csv file from varying sources (dependent on "style") and returns a pandas dataframe in a freezerworks-like format, minus the headers"""
    #We start each instance by reading in a file
    output_columns = [
        "(Neuro) Patient ID", "(HaflerLab) Substudy Visit", "(HaflerLab) Aliquot ID", 
        "(HaflerLab) Sample ID", "YCCI_Sample Type", "Sample Additive", 
        "Aliquot Type", "Sample Collection Date", "Aliquot Additive", 
        "Current Amount", "Aliquot UOM", "Sample Volume", "Sample UOM", "Notes"
    ]
    ONE_MILLION=1000000 #One million
    output_dataframe = pandas.DataFrame(columns=output_columns)
    if row_offset:
        current_dataframe = pandas.read_csv(input_dataframe_filepath+input_data_filename, header=row_offset) #could hypothetically be moved inside the parameters for each site
    else:
        current_dataframe = pandas.read_csv(input_dataframe_filepath+input_data_filename)
    if import_style=="DF":
        #make sure there are no completely empty rows
        #Now, parse information for the new Freezerworks style dataframe
        for idx, row in current_dataframe.iterrows(): 
            #Could one day even add an if statement up here for tumor samples if I deem it necessary
            if row.isnull().all() :
                continue #skip this row
            logging.debug(f"this loop runs, and we are on row \n {row} and index {idx}\n")
            output_row = pandas.Series()  # Temporary dictionary to store the transformed row
            # Populate each field according to the transformation logic

            # (Neuro) Patient ID
            if not pandas.isna(row["Patient ID (SitePatient Number_B_Timepoint)"]):
                neuro_patient_id = row["Patient ID (SitePatient Number_B_Timepoint)"][:2] + row["Patient ID (SitePatient Number_B_Timepoint)"][4:6] #extracts DF and ##, and then adds them
                output_row["(Neuro) Patient ID"] = neuro_patient_id
            else:
                logging.warning(f"Warning: value {row['Patient ID (SitePatient Number_B_Timepoint)']} detected in row {row}\n")
            
            # (HaflerLab) Substudy Visit
            row_value = row["Patient ID (SitePatient Number_B_Timepoint)"]
            if type(row_value) == str:
                visit_day=row_value[9:]
                if "EOT" in visit_day: #Highest priority
                    visit_day = "EOT"
                elif "SURG" in visit_day: #replace with OR
                    visit_day = "OR"
                elif visit_day[0]=="C": #replace with V... and remove D# part
                    visit_day = visit_day[:-2] #take everything but last two chars (which are always D1)
                    if len(visit_day)==2:
                        visit_day = "V0"+visit_day[1:]
                    else:
                        visit_day = "V"+visit_day[1:]
            output_row["(HaflerLab) Substudy Visit"] = visit_day
                
            #Also, there is no condition for PRE patients, because the syntax they already use is what we use
            #print(visit_day)
            #get everything after the second underscore
    
        
            # (HaflerLab) Aliquot ID (placeholder)
            output_row["(HaflerLab) Aliquot ID"] = None  # Modify with actual logic as needed
            
            # (HaflerLab) Sample ID (placeholder)
            output_row["(HaflerLab) Sample ID"] = None  # Modify with actual logic as needed
            #line from script iteration
            #haflerlab_sample_id= ''.join(str(num) for num in [ord(char) for char in (neuro_patient_id+haflerlab_substudy_visit+ycci_sample_type+sample_volume)])+"," #creates a unique sample ID based on the concatenation of patient ID, substudy visit, sample type, and sample volume
    
            # YCCI_Sample Type
            output_row["YCCI_Sample Type"] = "Blood"
            
            # Sample Collection Date
            output_row["Sample Collection Date"] = row["Date"]
            
            # Sample UOM
            output_row["Sample UOM"] = "mL"
            
            # Notes
            output_row["Notes"] = "Computer Generated Entry"
        
            #Update Sample ID based off ASCII information 
            #output_row["(HaflerLab) Sample ID"]=''.join(str(num) for num in [ord(char) for char in (output_row["(Neuro) Patient ID"]+output_row["(HaflerLab) Substudy Visit"]+output_row["YCCI_Sample Type"]+str(output_row["Sample Volume"]))]) #suffixes a number after the sample ID for the aliqout ID
    
            #For Serum Aliquots
            #temporarily assign sample volume to 6 and aliwuot type to Serum for Serum, then 25 and PBMC for PBMC after serum for loop is done
            output_row["Sample Volume"] = 6 #Sample UOM can be filled out earlier as it is mL in both cases
            output_row["Sample Additive"] = "No Additive"
            sample_id_string = (str(output_row["(Neuro) Patient ID"])+str(output_row["(HaflerLab) Substudy Visit"])+str(output_row["YCCI_Sample Type"])+str(output_row["Sample Volume"]))
            output_row["(HaflerLab) Sample ID"] = ''.join(str(num) for num in [ord(char) for char in sample_id_string])
            #print(f"Here is sample_id_string for Serum: {sample_id_string}")
            output_row["Aliquot Type"] = "Serum"   
            output_row["Aliquot Additive"] = "No Additive"
            output_row["Aliquot UOM"] = "mL"
            #output_row["(HaflerLab) Sample ID"]=''.join(str(num) for num in [ord(char) for char in (output_row["(Neuro) Patient ID"]+output_row["(HaflerLab) Substudy Visit"]+output_row["YCCI_Sample Type"]+str(output_row["Sample Volume"]))]) #suffixes a number after the sample ID for the aliqout ID
            number_of_serum_aliquots=row["Number of Serum Vials"]
            output_row = pandas.DataFrame(output_row) #These might work up here, otherwise they will have to move into the loops
            output_row=output_row.transpose()
            if not pandas.isna(number_of_serum_aliquots):
                output_row["Current Amount"] = .5
                for i in range(int(number_of_serum_aliquots)): 
                    logging.debug(f' Sample_id_string is: {sample_id_string} and its type is: {type(sample_id_string)}\nMeanwhile, output_row["(Neuro) Patient ID"]) is: {output_row["(Neuro) Patient ID"]}\noutput_row["(HaflerLab) Substudy Visit"] is: {output_row["(HaflerLab) Substudy Visit"]}\noutput_row["YCCI_Sample Type"] is: {output_row["YCCI_Sample Type"]}\noutput_row["Sample Volume"] is {output_row["Sample Volume"]}')
                    output_row["(HaflerLab) Aliquot ID"] = ''.join(str(num) for num in [ord(char) for char in sample_id_string])+"."+str(i)#suffixes a number after the sample ID for the aliqout ID
                    logging.debug(f'We are outputting aliquot number {i} with sample ID {output_row["(HaflerLab) Sample ID"]} and aliquot id {output_row["(HaflerLab) Aliquot ID"]}')
                    output_dataframe = pandas.concat([output_dataframe, output_row])
            
            #For PBMC Aliquots
            output_row["Sample Volume"] = 25  #Sample UOM can be filled out earlier as it is mL in both cases
            output_row["Sample Additive"] = "Lithium Heparin"
            output_row["Aliquot Type"] = "PBMC"
            output_row["Aliquot Additive"] = "DMSO"
            output_row["Aliquot UOM"] = "million"
            #output_row["(HaflerLab) Sample ID"]=''.join(str(num) for num in [ord(char) for char in (output_row["(Neuro) Patient ID"]+output_row["(HaflerLab) Substudy Visit"]+output_row["YCCI_Sample Type"]+str(output_row["Sample Volume"]))]) #suffixes a number after the sample ID for the aliqout ID
            sample_id_string = (str(output_row["(Neuro) Patient ID"].values[0])+str(output_row["(HaflerLab) Substudy Visit"].values[0])+str(output_row["YCCI_Sample Type"].values[0])+str(output_row["Sample Volume"].values[0]))
            logging.debug(f"Here is sample_id_string for PBMCs: {sample_id_string}")
            output_row["(HaflerLab) Sample ID"] = ''.join(str(num) for num in [ord(char) for char in sample_id_string])
            number_of_pbmc_aliquots=row["Number of PBMCs Vials"]
            # Current Amount
            logging.debug(f'This is {row["PBMC count (total cells/sample)"]} and this is wether it is pandas.isna(): {pandas.isna(row["PBMC count (total cells/sample)"])} and whether it is numpy.isnan(): {numpy.isnan(row["PBMC count (total cells/sample)"])}')
            if (not pandas.isna(number_of_pbmc_aliquots)) and (not pandas.isna(row["PBMC count (total cells/sample)"])):
                output_row["Current Amount"] = math.floor(int(row["PBMC count (total cells/sample)"]) / int(number_of_pbmc_aliquots)/ONE_MILLION) #if not pandas.isna(row["PBMC count (total cells/sample)"]) else print(f"Warning: NA value {row["PBMC count (total cells/sample)"]} in row {row}")
                for i in range(int(number_of_pbmc_aliquots)):
                    id_string = (str(output_row["(Neuro) Patient ID"].values[0])+str(output_row["(HaflerLab) Substudy Visit"].values[0])+str(output_row["YCCI_Sample Type"].values[0])+str(output_row["Sample Volume"].values[0]))
                    #output_row["(HaflerLab) Aliquot ID"] = ''.join(str(num) for num in [ord(char) for char in id_string])+"."+str(i)#suffixes a number after the sample ID for the aliqout ID
                    logging.debug(f'id_string is: {id_string} and its type is: {type(id_string)}\nMeanwhile, output_row["(Neuro) Patient ID"]) is: {output_row["(Neuro) Patient ID"]}\noutput_row["(HaflerLab) Substudy Visit"] is: {output_row["(HaflerLab) Substudy Visit"]}\noutput_row["YCCI_Sample Type"] is: {output_row["YCCI_Sample Type"]}\noutput_row["Sample Volume"] is {output_row["Sample Volume"]}')
                    output_row["(HaflerLab) Aliquot ID"] = ''.join(str(num) for num in [ord(char) for char in sample_id_string])+"."+str(i)#suffixes a number after the sample ID for the aliqout ID
                    logging.debug(f'We are outputting aliquot number {i} with sample ID {output_row["(HaflerLab) Sample ID"]} and aliquot id {output_row["(HaflerLab) Aliquot ID"]}')
                    output_dataframe = pandas.concat([output_dataframe, output_row])

        return output_dataframe 
    
    if import_style == "SF":
        #Remove rows with NA PBMC counts that represent the other PBMC samples via bitwise logic
        current_dataframe = current_dataframe.loc[~((current_dataframe["Cell Count"].isna()) & (current_dataframe["Sample Type"] == "PBMC"))]
        #do regex to convert scientific notation to python friendly version
        #in theory, this could be done inside the loop for efficiency, but that will come later
        current_dataframe["Cell Count"] = current_dataframe["Cell Count"].apply(lambda x: float(re.sub(r'.10\^', 'e', str(x))) if (isinstance(x, str) and ('X10^' or 'x10' in x)) else x)
        #Now, parse information for the new Freezerworks style dataframe
        for idx, row in current_dataframe.iterrows():
            #print(f"this loop runs, and we are on row \n {row} and index {idx}\n")
            output_row = pandas.Series()  # Temporary dictionary to store the transformed row
            # Populate each field according to the transformation logic
            
            # (Neuro) Patient ID
            neuro_patient_id = "SF" + row["Sample ID"][-2:] if isinstance(row["Sample ID"], str) else None
            output_row["(Neuro) Patient ID"] = neuro_patient_id
            
            # (HaflerLab) Substudy Visit
            visit_day = row.get("Visit Day", "")
            if "C" in visit_day:
                number=re.search("[0-9]+", visit_day)
                number=str(number.group()) #extracts the actual # from number and converts it to a str
                if len(number) ==1:
                    output_row["(HaflerLab) Substudy Visit"] = "V0" + visit_day[1]
                else:
                    output_row["(HaflerLab) Substudy Visit"] = "V" + visit_day[1]
                
            elif "PRE" in visit_day: #will have to handle later; current manifest does not have this so I don't know that their syntax is
                output_row["(HaflerLab) Substudy Visit"] = visit_day
            elif any(term in visit_day for term in ["OR", "SURG"]): #will have to handle later; current manifest does not have this so I don't know that their syntax is
                output_row["(HaflerLab) Substudy Visit"] = visit_day
            elif "Safety Follow Up" in visit_day:
                output_row["(HaflerLab) Substudy Visit"] = "FU01" #stands for follow up. For now, we only have 01.
            else:
                raise Exception(f"Invalid timepoint {visit_day} in Visit Day. Check CSV.")
        
            # (HaflerLab) Aliquot ID (placeholder)
            output_row["(HaflerLab) Aliquot ID"] = None  # Modify with actual logic as needed
            
            # (HaflerLab) Sample ID (placeholder)
            output_row["(HaflerLab) Sample ID"] = None  # Modify with actual logic as needed
            #line from script iteration
            #haflerlab_sample_id= ''.join(str(num) for num in [ord(char) for char in (neuro_patient_id+haflerlab_substudy_visit+ycci_sample_type+sample_volume)])+"," #creates a unique sample ID based on the concatenation of patient ID, substudy visit, sample type, and sample volume

            # YCCI_Sample Type
            output_row["YCCI_Sample Type"] = "Blood"
        
            # Sample Additive
            output_row["Sample Additive"] = "Lithium Heparin" if row["Sample Type"] == "PBMC" else "No Additive"
        
            # Aliquot Type
            output_row["Aliquot Type"] = row["Sample Type"]
        
            # Sample Collection Date
            output_row["Sample Collection Date"] = row["Sample Date"]
        
            # Aliquot Additive
            output_row["Aliquot Additive"] = "DMSO" if row["Sample Type"] == "PBMC" else "No Additive"
        
            # Current Amount
            output_row["Current Amount"] = math.floor(int(row["Cell Count"]) / int(row["Number of Aliquots Total"])/ONE_MILLION) if row["Sample Type"] == "PBMC" else 0.5
        
            # Aliquot UOM
            output_row["Aliquot UOM"] = "million" if row["Sample Type"] == "PBMC" else "mL"
        
            # Sample Volume
            output_row["Sample Volume"] = 25 if row["Sample Type"] == "PBMC" else 6
        
            # Sample UOM
            output_row["Sample UOM"] = "mL"
            
            # Notes
            output_row["Notes"] = "Computer Generated Entry"
        
            #Update Sample ID based off ASCII information 
            output_row["(HaflerLab) Sample ID"]=''.join(str(num) for num in [ord(char) for char in (output_row["(Neuro) Patient ID"]+output_row["(HaflerLab) Substudy Visit"]+output_row["YCCI_Sample Type"]+str(output_row["Sample Volume"]))]) #suffixes a number after the sample ID for the aliqout ID

            #convert series to DataFrame
            output_row=pandas.DataFrame(output_row)
            output_row=output_row.transpose()
        
            logging.debug(f"output row is: \n{output_row}\n")
            
            # Append the transformed row to Freezerworks_CSVsframe depending on the number of aliqouts with this information
            number_of_aliquots=row["Number of Aliquots Total"]
            for i in range(int(number_of_aliquots)):
                id_string = (str(output_row["(Neuro) Patient ID"].values[0])+str(output_row["(HaflerLab) Substudy Visit"].values[0])+str(output_row["YCCI_Sample Type"].values[0])+str(output_row["Sample Volume"].values[0]))
                logging.debug(f'id_string is: {id_string} and its type is: {type(id_string)}\nMeanwhile, output_row["(Neuro) Patient ID"]) is: {output_row["(Neuro) Patient ID"]}\noutput_row["(HaflerLab) Substudy Visit"] is: {output_row["(HaflerLab) Substudy Visit"]}\noutput_row["YCCI_Sample Type"] is: {output_row["YCCI_Sample Type"]}\noutput_row["Sample Volume"] is {output_row["Sample Volume"]}')
                output_row["(HaflerLab) Aliquot ID"] = ''.join(str(num) for num in [ord(char) for char in id_string])+"."+str(i)#suffixes a number after the sample ID for the aliqout ID
                output_dataframe = pandas.concat([output_dataframe, output_row])

        return output_dataframe



    if import_style=="FW_import": #there is no YU style because we don't have a manifest for ourselves. Instead it's just the freezerworks exports, which could come from any site
        #this one should be fairly easy because we're just re-importing the CSV
        output_columns= [
            "(Neuro) Patient ID", "(HaflerLab) Substudy Visit", "(HaflerLab) Aliquot ID", 
            "(HaflerLab) Sample ID", "YCCI_Sample Type", "Sample Additive", 
            "Aliquot Type", "Sample Collection Date", "Aliquot Additive", 
            "Current Amount", "Aliquot UOM", "Sample Volume", "Sample UOM", "Notes"
        ] #required to assign unique column names to sort 
        #Note, technically ones exported straight from FW will have the headers still in and an empty row in the indexes, so they should have headers and indices of 0      
        if(len(current_dataframe.columns) == len(output_columns)):
            current_dataframe.columns = output_columns
            return output_dataframe
        else:
            raise Exception(f"The number of columns in the old dataframe ({len(current_dataframe.columns)}) does not match the number of columns that is expected of this style ({len(output_columns)}).")    
 

def export_dataframe(input_dataframe: pandas.DataFrame, style:str="Fw_import", output_filepath:str="../data/exports/", output_filename:str=None, row_offset: Union[None,int]=None, column_offset: Union[None,int]=None) -> None:
    """Expects a dataframe in FW-import sytle, minus any headers or indices. Can either write it to a CSV for import into FW, or can generate a sample map CSV for use in the Hafler Lab sample map format. """
    
    if output_filename == None:
        output_filename=f"TEST_{style}_{datetime.now().strftime('%Y%m%d_%H%M%S')}" #.csv extension is appended later, once sample type is determined

    if style == "Fw_import":
        input_dataframe.to_csv(output_filepath, header=False, index=False) #To import into freezerworks the header and indices must be discarded
        return
    elif style == "Sample_map":
        input_dataframe["Internal Date"] = pandas.to_datetime(input_dataframe["Sample Collection Date"], format='%m/%d/%y')
        input_dataframe_sorted = input_dataframe.sort_values(by=["Internal Date", "(Neuro) Patient ID"], ascending=[True, True])
        serum_sample_map_dataframe=pandas.DataFrame()#(columns="Column 1")
        pbmc_sample_map_dataframe=pandas.DataFrame()
        serum_aliquots = pandas.Series()
        pbmc_aliquots = pandas.Series()
        for index, row in input_dataframe_sorted.iterrows():
            logging.debug(f"row is: {row} and index is: {index}")
            #Extract Relevant information
            #Patient ID
            patient_id = row["(Neuro) Patient ID"]
            logging.debug(f"This is patient_id: {patient_id}")
            #Timepoint
            timepoint = row["(HaflerLab) Substudy Visit"]
            logging.debug(f"This is timepoint: {timepoint}")
            #sample collection date
            date = row["Sample Collection Date"]
            logging.debug(f"This is date: {date}")
            #current amount
            current_amount = str(row["Current Amount"])
            logging.debug(f"This is current_amount: {current_amount}")
            #aliquot UOM
            aliquot_uom = row["Aliquot UOM"]
            logging.debug(f"This is aliquot_uom: {aliquot_uom}")
        
            entry_string = patient_id+timepoint+" "+date+" "+current_amount+aliquot_uom  
            logging.debug(f"This is entry_string: {entry_string}")
        
            
            if row["Aliquot Type"] == "PBMC":
        
                #add to PBMC Sample Series
                new_series = pandas.Series(entry_string)
                logging.debug(f"new_series is: {new_series}")
                pbmc_aliquots = pandas.concat([pbmc_aliquots, new_series])
                logging.debug(f"pbmc_aliquots is: {pbmc_aliquots}")
                #turn row into dataframe and transpose
                #then add to map
                #then,
                #maybe in here check to see if series length is 10, then write to dataframe, then reset?
                if len(pbmc_aliquots) == 10:
                    pbmc_aliquots = pandas.DataFrame(pbmc_aliquots)
                    pbmc_aliquots = pbmc_aliquots.transpose()
                    pbmc_sample_map_dataframe = pandas.concat([pbmc_sample_map_dataframe, pbmc_aliquots], ignore_index=True)
                    logging.debug(f"pbmc_sample_map_dataframe is: {pbmc_sample_map_dataframe}")
                    #now, reset pbmc_aliquots so it doesn't mess up the next version of the loop
                    pbmc_aliquots = pandas.Series()
            if row["Aliquot Type"] == "Serum":
        
                #add to serum Sample Series
                new_series = pandas.Series(entry_string)
                logging.debug(f"new_series is: {new_series}")
                serum_aliquots = pandas.concat([serum_aliquots, new_series])
                logging.debug(f"serum_aliquots is: {serum_aliquots}")
                #turn row into dataframe and transpose
                #then add to map
                #then,
                #maybe in here check to see if series length is 10, then write to dataframe, then reset?
                if len(serum_aliquots) == 10:
                    serum_aliquots = pandas.DataFrame(serum_aliquots)
                    serum_aliquots = serum_aliquots.transpose()
                    serum_sample_map_dataframe = pandas.concat([serum_sample_map_dataframe, serum_aliquots], ignore_index=True) #indexing error that I will have to address later
                    logging.debug(f"serum_sample_map_dataframe is: {serum_sample_map_dataframe}")
                    #now, reset serum_aliquots so it doesn't mess up the next version of the loop
                    serum_aliquots = pandas.Series()
        

        #Code to handle when the final row has less than 10 aliquots, in which case it will need padding with an empty string
        serum_aliquot_length_difference = 10-len(serum_aliquots)
        if serum_aliquot_length_difference > 0: #Checking if any difference exists
            extra_spaces = pandas.Series([" "] * serum_aliquot_length_difference)
            serum_aliquots = pandas.concat([serum_aliquots, extra_spaces], ignore_index=True)
        serum_aliquots = pandas.DataFrame(serum_aliquots)
        serum_aliquots = serum_aliquots.transpose()
        serum_aliquots.columns=[0]*10 # Sets all column names to 0, which allows for concatenation with the rest of the sample map dataframe
        serum_sample_map_dataframe = pandas.concat([serum_sample_map_dataframe, serum_aliquots], ignore_index=True) #this needs to run one more time after the loop is over so that the remaining samples are added

        pbmc_aliquot_length_difference = 10-len(pbmc_aliquots)
        if pbmc_aliquot_length_difference > 0: #Checking if any difference exists
            extra_spaces = pandas.Series([" "] * pbmc_aliquot_length_difference)
            pbmc_aliquots = pandas.concat([pbmc_aliquots, extra_spaces], ignore_index=True)
        pbmc_aliquots = pandas.DataFrame(pbmc_aliquots)
        pbmc_aliquots = pbmc_aliquots.transpose()
        pbmc_aliquots.columns=[0]*10 # Sets all column names to 0, which allows for concatenation with the rest of the sample map dataframe
        pbmc_sample_map_dataframe = pandas.concat([pbmc_sample_map_dataframe, pbmc_aliquots], ignore_index=True) #this needs to run one more time after the loop is over so that the remaining samples are added
        
        #writing to outputfile
        output_filename+="_serum.csv"
        serum_sample_map_dataframe.to_csv(output_filepath+output_filename, header=False, index=False)
        output_filename=output_filename[:-10] #remove last 10 chars, which was the previous ending
        output_filename+="_pbmc.csv"
        pbmc_sample_map_dataframe.to_csv(output_filepath+output_filename, header=False, index=False)


if __name__ == "__main__":
    main()